In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/datasetses/this_final.csv
/kaggle/input/datasetses/this_test.csv


Train and Test data were built by merging the files 
(calls.csv
customers.csv
reason.csv
sentiment_statistics.csv
test.csv) 
... and preprocessing them as discussed

In [2]:
train = pd.read_csv('/kaggle/input/datasetses/this_final.csv')
test = pd.read_csv('/kaggle/input/datasetses/this_test.csv')

In [5]:
train

,agent_id,elite_level_code,agent_tone,customer_tone,average_sentiment,silence_percent_average,primary_call_reason,waiting_time,handle_time,tfidf_mean_feature
0,963118,4.0,3,0,-0.04,0.39,Voluntary Cancel,7.0,31.0,0.011954
1,519057,0.0,1,3,0.02,0.35,Booking,3.0,12.0,0.011548
2,158319,0.0,3,4,-0.13,0.32,IRROPS,8.0,19.0,0.010403
3,488324,2.0,3,2,-0.20,0.20,Upgrade,5.0,7.0,0.011170
4,721730,0.0,3,4,-0.05,0.35,Seating,10.0,9.0,0.011489
...,...,...,...,...,...,...,...,...,...,...
66648,783441,2.0,3,2,-0.19,0.51,Mileage Plus,6.0,10.0,0.012907
66649,413148,0.0,1,1,0.05,0.34,Post Flight,9.0,28.0,0.011659
66650,980156,1.0,1,2,0.03,0.22,Upgrade,3.0,8.0,0.011966
66651,616988,0.0,1,4,0.05,0.42,Upgrade,6.0,6.0,0.010163


In [7]:
X = train.drop('primary_call_reason', axis = 1)
y = train['primary_call_reason']

In [8]:
from sklearn.preprocessing import *

In [9]:
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import log_loss, accuracy_score
from sklearn.preprocessing import LabelEncoder
from imblearn.combine import SMOTEENN
from imblearn.pipeline import make_pipeline
import numpy as np

le = LabelEncoder()
y_encoded = le.fit_transform(y)

base_estimator = DecisionTreeClassifier(max_depth=1)
model = AdaBoostClassifier(base_estimator=base_estimator, n_estimators=100, random_state=42)

smote_enn = SMOTEENN(random_state=42)
pipeline = make_pipeline(smote_enn, model)

kf = KFold(n_splits=10, shuffle=True, random_state=42)

predictions = cross_val_predict(pipeline, X, y_encoded, cv=kf, method='predict_proba')

loss = log_loss(y_encoded, predictions)

predicted_labels_encoded = np.argmax(predictions, axis=1)
predicted_labels = le.inverse_transform(predicted_labels_encoded)

accuracy = accuracy_score(y, predicted_labels)

print(f'Log Loss: {loss}')
print(f'Accuracy: {accuracy}')

/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/en

Log Loss: 2.433271427433663
Accuracy: 0.08695782635440265


In [13]:
np.unique(predicted_labels)

array(['Booking', 'Checkout', 'Digital Support', 'Upgrade'], dtype=object)

In [14]:
# Fit the pipeline on the entire training data
pipeline.fit(X, y_encoded)

# Predict probabilities on the test set
test_predictions_proba = pipeline.predict_proba(test)

# Get the predicted labels from the probabilities
test_predicted_labels_encoded = np.argmax(test_predictions_proba, axis=1)
test_predicted_labels = le.inverse_transform(test_predicted_labels_encoded)

# If you want the test predictions in a DataFrame
test_results_df = pd.DataFrame({
    'predicted_label': test_predicted_labels
})

/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


In [16]:
test['predicted_lebel'] = test_results_df

In [17]:
test

,agent_id,elite_level_code,agent_tone,customer_tone,average_sentiment,silence_percent_average,waiting_time,handle_time,tfidf_mean_feature,predicted_lebel
0,488324,0.0,3,3,-0.06,0.58,11.0,58.0,0.008770,Checkout
1,963118,0.0,1,0,0.01,0.40,8.0,13.0,0.011333,Digital Support
2,519057,0.0,3,1,0.00,0.49,10.0,2.0,0.010109,Digital Support
3,963118,0.0,1,0,0.02,0.28,4.0,4.0,0.011389,Booking
4,158319,3.0,3,2,-0.12,0.12,6.0,4.0,0.013353,Upgrade
...,...,...,...,...,...,...,...,...,...,...
5152,638327,0.0,3,4,-0.15,0.29,3.0,13.0,0.011043,Booking
5153,859203,2.0,1,2,0.03,0.67,8.0,63.0,0.011346,Digital Support
5154,520243,1.0,3,2,-0.14,0.02,5.0,5.0,0.011208,Booking
5155,255256,0.0,1,1,0.14,0.53,8.0,8.0,0.010844,Digital Support
